In [1]:
import numpy as np
np.random.seed(10)
from matplotlib import pyplot
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

wandb.login()

     |████████████████████████████████| 1.7 MB 7.9 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |████████████████████████████████| 181 kB 45.9 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
% cd /content/drive/MyDrive/github_dl_6910

/content/drive/MyDrive/github_dl_6910


In [6]:
from sklearn.metrics import accuracy_score

from assignment_1.ActivationLayerScalar import ActivationLayerScalar
from assignment_1.ActivationLayerFactory import tanh, tanh_d, sigmoid, sigmoid_d, softmax, softmax_d, relu, relu_d
from assignment_1.ActivationLayerVector import ActivationLayerVector
from assignment_1.Dense import Dense
from assignment_1.LossFactory import mse, mse_prime, cross_entropy, cross_entropy_d
from assignment_1.Network import Network
import numpy as np
from tensorflow.keras.datasets import fashion_mnist as mnist
from tensorflow.keras.utils import to_categorical




(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test_cat = to_categorical(y_test)


def train():


  default_configs = {"epochs": 3 ,
            "learning_rate": 0.001,
            "No_hidden_layer": 3,
            "Neurons": 32,
            "weight_decay": 0,
            "mini_batch_size": 64,
            "weight_initialization": 'xavier',
            "activation_type": "relu",
            "loss_optimizer_type": "adam",
            }

  run = wandb.init(project='CS6910_assignment1', config=default_configs)
  config = wandb.config

  # Network
  nw = Network()
  nw.use(mse, mse_prime, optimizer=config.loss_optimizer_type, learning_rate=config.learning_rate)
  nw.add(Dense(28*28, config.Neurons, initializer_type=config.weight_initialization))
  nw.add(ActivationLayerScalar(activation=config.activation_type))
  nw.add(Dense(config.Neurons, config.Neurons, initializer_type=config.weight_initialization))
  nw.add(ActivationLayerScalar(activation=config.activation_type))
  nw.add(Dense(config.Neurons, 10, initializer_type=config.weight_initialization))
  nw.add(ActivationLayerVector(softmax, softmax_d))


  history_list = nw.fit(x_train, y_train, x_test, y_test_cat, epochs=config.epochs, batch_size=config.mini_batch_size)

  for i, test_acc, err, train_acc in history_list:
    wandb.log({"epoch" : i,
    "test_accuracy" : test_acc,
    "train_loss": err,
    "train_accuracy":train_acc})


  y_test_pred = nw.predict(x_test)
  test_accuracy = accuracy_score(y_test, np.squeeze(y_test_pred))

  print("\n")
  print('Test Set Score ', test_accuracy)

In [7]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [5,8]
        },
        'No_hidden_layer': {
            'values': [3,4,5]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4, 1e-2]
        },'Neurons':{
            'values': [32,64,128]
        },
        'weight_decay': {
            'values': [0,0.0005,0.5]
        },'mini_batch_size':{
            'values': [16,32,64]
        },'loss_optimizer_type':{
            'values': ['rmsprop','adam','nadam', 'SGD', 'Momentum']
        },'weight_initialization': {
            'values': ['random','xavier']
        },'activation_type':{
            'values': ['sigmoid','tanh', 'relu']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910_Assignment_I")
wandb.agent(sweep_id, train,count = 20)

Create sweep with ID: 84ksa7yb
Sweep URL: https://wandb.ai/adi8196/CS6910_Assignment_I/sweeps/84ksa7yb


wandb: Agent Starting Run: 4qzo8ncb with config:
wandb: 	Neurons: 64
wandb: 	No_hidden_layer: 4
wandb: 	activation_type: tanh
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	loss_optimizer_type: nadam
wandb: 	mini_batch_size: 16
wandb: 	weight_decay: 0.5
wandb: 	weight_initialization: xavier
wandb: Currently logged in as: adi8196 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


100%|██████████| 60000/60000 [00:48<00:00, 1247.93it/s]


epoch 1/5   error=0.023563
epoch 1/5   train_accuracy=0.859883
epoch 1/5   test_accuracy=0.849300


100%|██████████| 60000/60000 [00:50<00:00, 1194.52it/s]


...

epoch 3/8   error=0.025726
epoch 3/8   train_accuracy=0.826483
epoch 3/8   test_accuracy=0.813400


100%|██████████| 60000/60000 [01:26<00:00, 693.38it/s]


epoch 4/8   error=0.023970
epoch 4/8   train_accuracy=0.837317
epoch 4/8   test_accuracy=0.820300


100%|██████████| 60000/60000 [01:25<00:00, 698.57it/s]


epoch 5/8   error=0.022660
epoch 5/8   train_accuracy=0.844750
